In [1]:
import cv2
import numpy as np

In [2]:
lane  = cv2.VideoCapture(r"C:\Users\DELL\Downloads\lane.mp4")

In [3]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

In [4]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters [0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
        print(left_fit_average, 'left')
        left_line = make_coordinates(image, left_fit_average)
    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        print(right_fit_average, 'right')
        right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

In [5]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

In [6]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0), 10)
    return line_image

In [7]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
    [(200,height), (1100,height), (550,250)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [8]:
while(lane.isOpened()):
    _, frame = lane.read()
    canny_image = canny(frame)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow("result", combo_image)
    cv2.waitKey(1)

[  -1.87340799 1356.76218709] left
[   0.96951501 -269.81509634] right
[  -1.88435374 1359.94557823] left
[   0.97471477 -275.61410206] right
[  -1.78200297 1302.18676705] left
[   0.96642222 -266.58775585] right
[  -1.78200297 1302.18676705] left
[   0.96642222 -266.58775585] right
[  -1.88206973 1356.38840181] left
[   0.97484009 -274.8739918 ] right
[  -1.77854887 1294.98519468] left
[   0.97063712 -271.84389211] right
[  -1.77854887 1294.98519468] left
[   0.97063712 -271.84389211] right
[  -1.76630363 1292.859795  ] left
[   0.97041079 -269.91937796] right
[  -1.76630363 1292.859795  ] left
[   0.97041079 -269.91937796] right
[  -1.87712447 1351.57067535] left
[   0.98154688 -274.37885807] right
[  -1.87712447 1351.57067535] left
[   0.98154688 -274.37885807] right
[  -1.87413206 1349.75890004] left
[   0.97164537 -268.29866335] right
[  -1.88740322 1355.0395802 ] left
[   0.9793419 -272.5943296] right
[  -1.88740322 1355.0395802 ] left
[   0.9793419 -272.5943296] right
[  -1.7367

UnboundLocalError: local variable 'left_line' referenced before assignment